In [1]:
!pip install tensorflow
!pip install keras
!pip install pandas
!pip install hazm

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [2]:
import pandas as pd
import numpy as np
from hazm import *
import re
import tensorflow as tf
from keras.models import Sequential, save_model
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib as mpl
import numpy as np
import math
import re
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sb
from mpl_toolkits.axes_grid1 import make_axes_locatable
from copy import deepcopy
from string import punctuation
import random

2022-07-15 09:31:51.269881: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-07-15 09:31:51.269940: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
dataset = pd.read_csv("hamshahri.csv")
print(dataset.head())

      DID      Date    Cat                                               Text
0     1S1  75\04\02  adabh  جاودانگي در زندگي گروهي از طريق هنر \nنگاهي به...
1  DID2S1  75\04\02  adabh  رويدادهاي هنري جهان \nنمايشگاه هنر در خدمت ديك...
2  DID3S1  75\04\02  adabh  برديوار نگارخانه ها \nگالري گلستان: \nنمايشگاه...
3  DID4S1  75\04\02  ejtem  بازي را جدي بگيريم \nمطالعه اي مقدماتي پيرامون...
4  DID5S1  75\04\02  elmfa  تخته سياه و غباري كه سترده نمي شود... \nاشاره;...


In [4]:
data_len = dataset.shape[0]
print('Dataset has {} rows.'.format(data_len))

Dataset has 165233 rows.


In [5]:
cats = dataset["Cat"]
texts = dataset["Text"]
unique_cats = np.unique(cats)
cat_texts_count = dict.fromkeys(unique_cats, 0)

for cat in cats:
  cat_texts_count[cat] += 1

for cat, cnt in cat_texts_count.items():
  print('Category "{}" contains {} posts.'.format(cat, cnt))

Category ". راسته: . شب كلاه شكلان (Semaeostomae)" contains 1 posts.
Category "Adabh" contains 13 posts.
Category "Aeqts" contains 10 posts.
Category "Akhar" contains 10 posts.
Category "Busiw" contains 9 posts.
Category "Cinew" contains 3 posts.
Category "Cultw" contains 651 posts.
Category "Econo" contains 6 posts.
Category "Eqtes" contains 14 posts.
Category "Globa" contains 4 posts.
Category "Gozar" contains 3 posts.
Category "Kharj" contains 22 posts.
Category "Lifew" contains 3 posts.
Category "Musical" contains 3 posts.
Category "Newsp" contains 9 posts.
Category "Nnaft" contains 6 posts.
Category "Polig" contains 2 posts.
Category "Santj" contains 4 posts.
Category "Sciew" contains 6 posts.
Category "Shari" contains 8 posts.
Category "Siasi" contains 18 posts.
Category "Sporw" contains 50 posts.
Category "Theatre" contains 3 posts.
Category "Thought" contains 2 posts.
Category "abksh" contains 495 posts.
Category "adabh" contains 5171 posts.
Category "adarman" contains 31 posts

In [6]:
with open ("fa_stop_words.txt", "r") as fa_stop_words:
    stopwords = fa_stop_words.read().splitlines()

In [7]:
normalizer = Normalizer()
lemmatizer = Lemmatizer()
stemmer = Stemmer()

def preprocessing(text):
    text = re.sub('<[^<]+?>','', text)
    text = ''.join(c for c in text if not c.isdigit())
    text = normalizer.normalize(text)
    text = ' '.join(word for word in text.split() if word not in stopwords)
    return text

## PreProcess

In [8]:
num_classes = len(unique_cats)

MAX_NB_WORDS = 50000
MAX_SEQUENCE_LENGTH = 250
EMBEDDING_DIM = 100

tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(dataset["Cat"].values)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

X = tokenizer.texts_to_sequences(dataset["Cat"].values)
X = tf.keras.preprocessing.sequence.pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X.shape)

Found 155 unique tokens.
Shape of data tensor: (165233, 250)


In [44]:
rows_of_train = 250000
rows_of_test = 75000

Y_for_predict = pd.get_dummies(dataset['Cat'])
Y = Y_for_predict.values
Y_for_predict = Y_for_predict.columns.values
print('Shape of label tensor:', Y.shape)

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3)

print(X_train.shape,y_train.shape)
print(X_test.shape,y_test.shape)

Shape of label tensor: (165233, 113)
(115663, 250) (115663, 113)
(49570, 250) (49570, 113)


In [46]:
Y_for_predict

array(['. راسته: . شب كلاه شكلان (Semaeostomae)', 'Adabh', 'Aeqts',
       'Akhar', 'Busiw', 'Cinew', 'Cultw', 'Econo', 'Eqtes', 'Globa',
       'Gozar', 'Kharj', 'Lifew', 'Musical', 'Newsp', 'Nnaft', 'Polig',
       'Santj', 'Sciew', 'Shari', 'Siasi', 'Sporw', 'Theatre', 'Thought',
       'abksh', 'adabh', 'adarman', 'aeqts', 'akhar', 'art', 'artw',
       'axrooz', 'bankb', 'bazar', 'books', 'busiw', 'cartoon', 'cinama',
       'cinew', 'city', 'cultw', 'donya', 'earth', 'econo', 'econw',
       'ejtem', 'elmfa', 'elmif', 'eqtes', 'eqtsj', 'ertebat', 'erteg',
       'gards', 'globa', 'gofgu', 'goftg', 'gozar', 'gqarn', 'gungn',
       'hamln', 'havad', 'igozar', 'ikabar', 'imaqal', 'infor', 'intep',
       'jjahn', 'jvarz', 'kharj', 'lastp', 'lifew', 'lite', 'maqal',
       'maref', 'media', 'mohit', 'mskan', 'musical', 'nameh', 'newsp',
       'nnaft', 'norooz', 'polig', 'sanat', 'santj', 'scien', 'sciew',
       'shahr', 'shahz', 'shari', 'shora', 'shrst', 'siasi', 'socie',
       

## Classification

In [10]:
model = Sequential()
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X.shape[1]))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(113, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

epochs = 2
batch_size = 64
history = model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=epochs,batch_size=batch_size)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   


2022-07-15 09:32:05.458105: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-07-15 09:32:05.458141: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-07-15 09:32:05.458177: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (milad): /proc/driver/nvidia/version does not exist
2022-07-15 09:32:05.458849: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


 embedding (Embedding)       (None, 250, 100)          5000000   
                                                                 
 spatial_dropout1d (SpatialD  (None, 250, 100)         0         
 ropout1D)                                                       
                                                                 
 lstm (LSTM)                 (None, 100)               80400     
                                                                 
 dense (Dense)               (None, 113)               11413     
                                                                 
Total params: 5,091,813
Trainable params: 5,091,813
Non-trainable params: 0
_________________________________________________________________
Epoch 1/2
1808/1808 [==============================] - 669s 369ms/step - loss: 0.4546 - accuracy: 0.9168 - val_loss: 0.0335 - val_accuracy: 0.9953
Epoch 2/2
1808/1808 [==============================] - 3959s 2s/step - loss: 0.0257 - accuracy: 0.9958 - val_loss: 0.

In [11]:
Y_pred = model.predict([X_test])

1550/1550 [==============================] - 136s 88ms/step


In [47]:
def prediction(text):
    #_text = preprocessing(text)
    _text = tokenizer.texts_to_sequences([text])
    _text = tf.keras.preprocessing.sequence.pad_sequences(_text, maxlen=MAX_SEQUENCE_LENGTH)
    result = model.predict([_text])[0]
    print(result)
    _max = max(result)
    index = np.where(result == _max)[0][0]
    print(index)
    cat = Y_for_predict[index]
    print("Category is {} with {} probability.".format(cat, _max))

prediction("رويدادهاي هنري جهان \nنمايشگاه هنر در خدمت ديكتاتورها \nنمايشگاه ارتباط بين هنر وقدرت ديكتاتورها در اروپا درشهر برلين \nآغاز به كار كرد. به گزارش واحد مركزي خبر به نقل از شبكه خبري ")

1/1 [==============================] - 0s 66ms/step
[0.00106386 0.00228523 0.00749621 0.00561704 0.00323148 0.00064356
 0.00252849 0.00748603 0.0033048  0.00091484 0.00102096 0.00118902
 0.00182613 0.00087702 0.01192232 0.00318546 0.00165332 0.00244924
 0.00681546 0.00590535 0.00334159 0.00168766 0.00332212 0.00159921
 0.0018912  0.00144132 0.00655162 0.00333786 0.0272177  0.00318568
 0.0043675  0.00203117 0.00826713 0.00186589 0.00183371 0.00265081
 0.00243636 0.06304602 0.02012181 0.00135081 0.0024714  0.00655597
 0.00270285 0.01038769 0.0007341  0.04962467 0.00050842 0.0035104
 0.00759989 0.04031919 0.02780611 0.06173796 0.0125184  0.0584808
 0.01004761 0.00466112 0.00353309 0.00400448 0.00756399 0.00389807
 0.02595487 0.00057444 0.00801433 0.00805908 0.00620531 0.0010234
 0.0045727  0.00119307 0.00156484 0.00129784 0.01909471 0.01371987
 0.00320766 0.00388127 0.01714039 0.00477145 0.00137795 0.01041315
 0.01369796 0.00450961 0.00295535 0.03260589 0.00369096 0.00362021
 0.00543483 0